In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import imblearn
import xgboost
import lightgbm as lgb
import torch
import datetime as dt
import random
import time
import os
import pickle
import ast
import dataframe_image as dfi

import matplotlib.lines as mlines
from matplotlib.patches import Rectangle

from scipy.interpolate import make_interp_spline, BSpline

from sklearn.metrics import make_scorer

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import torch.nn.functional as F
from skorch import NeuralNetClassifier

from imblearn.pipeline import Pipeline

from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

In [2]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"

In [3]:
palette = sns.color_palette([
    '#4682b4', '#AF4035', '#00937F', '#FFA500', '#7B5141',
    '#B86A84', '#5B3256', '#2E3876', '#587B7F', '#006D6F',
    '#A7CBE8', '#6D9BC3', '#4D7DA8', '#3D6384', '#304E68',
    '#AAAAAA', '#808080', '#707070', '#606060', '#505050'
])

corr_matrix = sns.color_palette('blend:#5691C4,#FFFFFF,#BA160C', as_cmap=True)
report = sns.color_palette('blend:#E6E9EC,#FFFFFF,#9E2A2B', as_cmap=True)
gradient = sns.color_palette('blend:#FFFFFF,#3D6384', as_cmap=True)
pairplot = sns.color_palette(['#8DB4D7', '#AF4035'])

In [4]:
# 1 2 3 4 5
sns.color_palette(palette[:5])

[(0.27450980392156865, 0.5098039215686274, 0.7058823529411765),
 (0.6862745098039216, 0.25098039215686274, 0.20784313725490197),
 (0.0, 0.5764705882352941, 0.4980392156862745),
 (1.0, 0.6470588235294118, 0.0),
 (0.4823529411764706, 0.3176470588235294, 0.2549019607843137)]

In [5]:
# 6 7 8 9 10
sns.color_palette(palette[5:10])

[(0.7215686274509804, 0.41568627450980394, 0.5176470588235295),
 (0.3568627450980392, 0.19607843137254902, 0.33725490196078434),
 (0.1803921568627451, 0.2196078431372549, 0.4627450980392157),
 (0.34509803921568627, 0.4823529411764706, 0.4980392156862745),
 (0.0, 0.42745098039215684, 0.43529411764705883)]

In [15]:
# -10 -9 -8 -7 -6
sns.color_palette(palette[10:15])

[(0.6549019607843137, 0.796078431372549, 0.9098039215686274),
 (0.42745098039215684, 0.6078431372549019, 0.7647058823529411),
 (0.30196078431372547, 0.49019607843137253, 0.6588235294117647),
 (0.23921568627450981, 0.38823529411764707, 0.5176470588235295),
 (0.18823529411764706, 0.3058823529411765, 0.40784313725490196)]

In [16]:
# -5 -4 -3 -2 -1
sns.color_palette(palette[15:])

[(0.6666666666666666, 0.6666666666666666, 0.6666666666666666),
 (0.5019607843137255, 0.5019607843137255, 0.5019607843137255),
 (0.4392156862745098, 0.4392156862745098, 0.4392156862745098),
 (0.3764705882352941, 0.3764705882352941, 0.3764705882352941),
 (0.3137254901960784, 0.3137254901960784, 0.3137254901960784)]

In [8]:
custom_params = {
    
    'figure.figsize': (8, 4), # ширина и высота в дюймах
    'axes.edgecolor': '0.75',
    'axes.titlepad': 10, # расстояние между заголовком и графиком
    'axes.titlesize': 10, # заголовок
    'axes.titleweight': 'bold', # заголовок
    'axes.labelsize': 10, # оси
    'axes.labelweight': 'normal', # подписи осей
    'axes.labelcolor': '0.3', # цвет подписей осей
    'axes.labelpad': 8, # расстояние между подписью оси и осью
    'xtick.labelsize': 9, # деления оси X
    'xtick.color': '0.35', # цвет подписей оси X
    'ytick.labelsize': 9, # деления оси Y
    'ytick.color': '0.35', # цвет подписей оси Y
    'axes.spines.left': True, # не отображать ось Y
    'axes.spines.right': False, # не отображать правую рамку 
    'axes.spines.top': False, # не отображать верхнюю рамку
    'axes.spines.bottom': True, # отображать нижнюю рамку
    'lines.markersize': 4, # размер маркеров
    'grid.color': '0.3',
    'grid.linestyle': ':',
    'grid.linewidth': 0.1,
    'lines.linewidth': 1.2,
}

sns.set_theme(style='whitegrid', palette=palette, rc=custom_params)

pd.set_option('display.max_rows', 100) # максимальное число отображаемых строк в head)
pd.set_option('display.max_columns', 20) # число отображаемых столбцов по умолчанию (здесь 20)

# Если необходимо посмотреть определенное число строк в датафрейме (здесь 25):
# with pd.option_context('display.max_rows', 10): display(df.head(25))

%matplotlib inline

# seed = 0
seed = 42